#Recolectar y preparar datos sobre Naves Imperiales de Star Wars


In [1]:
#@title Librerías

import sys, os, re, random
import urllib, time, base64
import urllib.parse
import string
import requests
import numpy as np

from google.colab import files


print("Librerías cargadas.")

Librerías cargadas.


#Recolectar Datos de Naves Imperiales

Fuente de datos: http://insd.swcombine.com/insd.htm

Atributos disponibles:
*  Model: The ship's class, name or model number.
*  Manufacturer: The ship's manufacturer or designer.
*  Designation: The ship's combat designation or classification.
* Length: The ship's length measured in meters.
* Crew: The ship's total number of crew members - excluding droids.
* Troops: The ship's total number of troops or passengers it can carry.
* Cargo Capacity: The ship's cargo capacity measured in kilograms or metric tons.
* Consumables: How long time the ship can operate without stopping to refuel or restock stores of food, water or air.
* Hyperdrive Multiplier: The ship's main hyperdrive class rating. This measures how quickly the ship travels a hyperspace route.
* Hyperdrive Backup: The ship's backup hyperdrive class rating. This measures how quickly the ship travels a hyperspace route if its main hyperdrive is disabled.
* Speed: The ship's sublight speed rating measured in MegaLights (MGLT).
* Hull: The ship's hull/armor durability rating measured in Resistance Units (RU).
* Shields: The ship's deflector shield durability rating measured in Shield Base Durability (SBD).
* Special Features: The ship's special design features.
* Weapons: The ship's standard armament listing.
* Onboard Craft: A listing of the ship's onboard support vessels. The listing is excluding all Barges and Tugs.

In [2]:
#@title Extraer datos disponibles en la página

# funciones para manejar campos pelicula

def strbuscarEntre(text, key1, key2):
  return strbuscarEntreList(text, [key1], [key2])

def strbuscarEntreList(text, listkey1, listkey2):
 p1 = -1
 for key1 in listkey1:
  auxP1 = text.find(key1)
  if (auxP1 > -1) and ((p1 == -1) or (auxP1 < p1)):
    p1 = auxP1
 if p1 == -1:
  return ""
 else:
  p1 = p1+len(key1)
  p2 = -1
  for key2 in listkey2:
    auxP2 = text[p1:].find(key2)
    if (auxP2 > -1) and ((p2 == -1) or (auxP2 < p2)):
      p2 = auxP2
  if p2 == -1:
    return ""
  else:
    p2 = p1+p2
  return text[p1:p2]

def limpiar(text):
  if text is None:
    return ""
  else:
    text = text.replace("&quot;", " ")
    text = text.replace("&amp;", " & ")
    text = text.replace("&nbsp;", "")
    text = text.replace("\r\n", "")
    text = text.replace("  ", " ")
    text = text.replace("  ", " ")
    return text.strip()


class INSDParser:

 def __init__(self, baseURL="http://insd.swcombine.com/insd/", debug=False):
  self.__debug = debug
  self.__baseURL = baseURL
  self.__AttNames = ["Model", "Manufacturer", "Length", "Crew", "Troops", "Cargo Capacity", "Consumables", "Hyperdrive Multiplier", "Hyperdrive Backup", "Speed", "Hull", "Shields", "Special Features", "Weapons", "Onboard Craft"]
  print("\n** Parser de 'IMPERIAL NAVY SHIP DATABASE' incializado ** \n")

 def __getHTMLContent(self, url):
  return requests.get(url).text

 def extractSiteLinks(self, searchURL):
  # ejecuta y obtiene la página
  searchData = self.__getHTMLContent(searchURL)
  # extrae los links
  auxList = re.findall('<A HREF=\"([\-\_a-zA-Z0-9]+\.htm)', searchData)
  if auxList is None:
    if self.__debug:
      print("\t -- No se encuentra links de Naves!!!")
    return None
  # saca duplicados
  linksList = list(set(auxList))
  if self.__debug:
    print("\t = lista de links (", len(linksList), "): ", linksList)
  # devuelve links de obtenidos
  return linksList, searchData

 def fetchNavesInfo(self, url, shipListData):
  # carga los datos de la página
  pagedData = self.__getHTMLContent(self.__baseURL + url)
  # obtiene datos
  matchStr = strbuscarEntre(pagedData, '<BLOCKQUOTE>', '</BLOCKQUOTE>')
  if (matchStr is None) or (matchStr == ""):
    print("\t -- No se puede encontrar datos en ", url)
    return None
  # extrae datos de la página
  naveData = {}
  # agrega nombre
  naveData["Name"] = limpiar(strbuscarEntre(pagedData, '<TITLE>', '</TITLE>'))
  # extra tipo de nave de la lista
  naveData["Ship Type"] = "???"
  posName = shipListData.find(url)
  if posName > 0:
      auxData = shipListData[:posName]
      posType = auxData.rfind('<B>')
      if posType > 0:
        auxData = shipListData[posType-1:posName]
        naveData["Ship Type"] = strbuscarEntre(auxData, '<B>', '</B>')
  # extrae lista atributos
  for att in self.__AttNames:
    auxStr = strbuscarEntre(matchStr, att+":", "<")
    if auxStr != "":
      naveData[att] = auxStr
  # limpia datos extraidos
  for k in naveData.keys():
    naveData[k] = limpiar(naveData[k])
  # agrega links
  naveData["URL_data"] =  self.__baseURL + url
  naveData["URL_image"]  = self.__baseURL + 'Ship_files/' + strbuscarEntre(pagedData, '<IMG SRC=\"Ship_files/', '"')
  return naveData


 def fetchAllNavesInfo(self, url="link.htm"):
  # carga los datos de la página
  linksList, shipListData = self.extractSiteLinks( self.__baseURL + url)
  if linksList is None:
    return None
  navesList = []
  for urlNave in linksList:
    if urlNave not in ["other2.htm", "techspec.htm"]:
      # busca la información de la nave
      naveInfo = self.fetchNavesInfo(urlNave, shipListData)
      if naveInfo is not None:
        navesList.append(naveInfo)
  return navesList


mostar_detalle = True #@param {type:"boolean"}

# busca y recolecta
parser = INSDParser(debug=mostar_detalle)
navesList = parser.fetchAllNavesInfo()

# muestra
if navesList is not None:
  print("")
  print("Se recolectaron ", len(navesList), " naves.")
  print("")
  if mostar_detalle:
    for nave in navesList:
      for k in nave.keys():
        print("  " + k + " = " + nave[k])
      print("")


** Parser de 'IMPERIAL NAVY SHIP DATABASE' incializado ** 

	 = lista de links ( 109 ):  ['gunship.htm', 'tiebt.htm', 'adzclass.htm', 'ipv1.htm', 'tieadvanc.htm', 'eclip.htm', 'kappa.htm', 'lamshut.htm', 'a9vigil.htm', 'toscanf.htm', 'sfslps.htm', 'builder.htm', 'tielone.htm', 'tieexm4.htm', 'tieoppr.htm', 'strike.htm', 'atshut.htm', 'assincor.htm', 'xmmiss.htm', 'esccar.htm', 'escshut.htm', 'dsd.htm', 'tiex1.htm', 'isp.htm', 'tiev38.htm', 'tiesub2.htm', 'ssd.htm', 'tieexm3.htm', 'tiephantom.htm', 'tiebomber.htm', 'tiex2.htm', 'decimator.htm', 'taskcru.htm', 'vensd.htm', 'tieaggre.htm', 'demolish.htm', 'enforcer.htm', 'deltrn1.htm', 'isd1.htm', 'isd2.htm', 'stargal.htm', 'gat.htm', 'tiehbomb.htm', 'tieinter.htm', 'tiex3.htm', 'tierpt2.htm', 'tiefc.htm', 'sover.htm', 'assshut.htm', 'tie.htm', 'tiefighter.htm', 'asstrn.htm', 'i7howlrun.htm', 'tiestrcr.htm', 'nebulonb.htm', 'tiehunt.htm', 'vsd1.htm', 'galleon.htm', 'tiegt.htm', 'tiest.htm', 'esctrn.htm', 'vsd2.htm', 'tiescimi.htm', 'sent

#Revisar Datos de Naves Imperiales

In [3]:
#@title Crear un DataFrame de los datos recolectados
import pandas as pd

df = pd.DataFrame.from_dict(navesList)

display( df )

,Name,Ship Type,Model,Manufacturer,Length,Crew,Cargo Capacity,Consumables,Hyperdrive Multiplier,Hyperdrive Backup,Speed,Hull,Shields,Weapons,URL_data,URL_image,Troops,Onboard Craft,Special Features
0,Corellian Gunship,Medium Ships,Corellian Gunship,Corellian Engineering Corporation,119 meters,91,300 metric tons,8 months,x2,x16,29 MGLT,176 RU,416 SBD,"8 Double Turbolaser Cannons, 6 Quad Laser Cann...",http://insd.swcombine.com/insd/gunship.htm,http://insd.swcombine.com/insd/Ship_files/CORG...,NaN,NaN,NaN
1,TIE Bomber Trainer,TIE Support Craft,TIE/bmt Bomber Trainer,Sienar Fleet Systems,7.8 meters,1,15 metric tons (bomb bay),1 day,NaN,NaN,80 MGLT,28 RU,NaN,"2 Laser Cannons, 1 Concussion Missile Launcher...",http://insd.swcombine.com/insd/tiebt.htm,http://insd.swcombine.com/insd/Ship_files/TIEB...,1 (trainer or priority personnel),NaN,NaN
2,Adz Class Destroyer,Medium Ships,Ads Class Destroyer,Imperial Shipyards,150 meters,36,400 metric tons,6 months,x0.55,x12,83 MGLT,162 RU,600 SBD,3 Quad Laser Cannon and 3 Ion Cannon Batteries.,http://insd.swcombine.com/insd/adzclass.htm,http://insd.swcombine.com/insd/Ship_files/NOPI...,8,6 TIE Series Fighters.,NaN
3,IPV 1 System Patrol Craft,Medium Ships,IPV 1 System Patrol Craft,Sienar Fleet Systems,153 meters,12,200 metric tons,3 months,NaN,NaN,78 MGLT,285 RU,400 SBD,4 Double Turbolaser Cannons,http://insd.swcombine.com/insd/ipv1.htm,http://insd.swcombine.com/insd/Ship_files/IPV1...,10,NaN,NaN
4,TIE Advanced,TIE Fighters,TIE/ad Fighter,Sienar Fleet Systems,7.8 meters,1,80 kilograms,5 days,x2,NaN,133 MGLT,14 RU,40 SBD,4 Laser Cannons and 2 General Purpose Launchers.,http://insd.swcombine.com/insd/tieadvanc.htm,http://insd.swcombine.com/insd/Ship_files/TIEA...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,Acclamator Class Transport Ship,Heavy Ships,Acclamator Class Transport,Rothana Heavy Engineering,752 meters,700,"15,000 metric tons",1 year,x0.6,x20,8 MGLT,"1,240 RU","2,720 SBD","12 Quad Turbolaser Turrets, 24 Laser Cannons a...",http://insd.swcombine.com/insd/Acclamator.htm,http://insd.swcombine.com/insd/Ship_files/Accl...,"16,000",72 TIE Series Fighters and 4 Light Transports.,Planetary Flight Capability
103,Preybird Class Starfighter,Other Starfighters,Preybird Class Starfighter,SoroSuub Corporation / Cavrilhu Shipyards,21 meters,1,15 kilograms,4 days,x3,NaN,100 MGLT,18 RU,46 SBD,2 Laser Cannons and 2 Concussion Missile Launc...,http://insd.swcombine.com/insd/preybird.htm,http://insd.swcombine.com/insd/Ship_files/PREY...,NaN,NaN,NaN
104,Svelte Class Imperial Shuttle,Shuttles,Svelte Class Imperial Shuttle,Silviut Corporation,17 meters,6,50 metric tons,2 months,x1,x5,60 MGLT,53 RU,100 SBD,2 Laser Cannons and 2 Double Blaster Cannons.,http://insd.swcombine.com/insd/svelte.htm,http://insd.swcombine.com/insd/Ship_files/SVEL...,15,NaN,Adjustable stabilizer fins.
105,Vibre Class Assault Cruiser,Medium Ships,Vibre Class Assault Cruiser,Silviut Corporation,100 meters,45,500 metric tons,5 months,x1,x15,70 MGLT,257 RU,622 SBD,4 Laser Cannons Turrets and 4 Ion Cannons and ...,http://insd.swcombine.com/insd/vibre.htm,http://insd.swcombine.com/insd/Ship_files/VIBR...,60,NaN,NaN


In [4]:
#@title Mostrar Estadísticas de datos recolectados


# variables auxiliares
atributo_clase = ""

# configura para que muestre todas las columnas y filas
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# devuelve listas de columnas numéricas y no numéricas
def devolNombreColumnas(ndf):
  colValues = []
  colNoValues = []
  for col in ndf.columns:
    if ndf[col].dtypes in ("object", "bool"):
      colNoValues.append( col )
    else:
      colValues.append( col )
  return colValues, colNoValues

# función auxiliar para separar datos de entrada y de salida
def separarDatosXY(ndf, atributo_clase="", xSoloNros=True):
  # hace una copia auxiliar del data frame
  cdf = ndf.copy()
  # saca el atributo clase (OPCIONAL)
  if atributo_clase == "":
    Y = []
  else:
    # datos atributo clase
    Y = np.array( cdf.pop(atributo_clase).fillna("-NAN-") )
  if xSoloNros:
    # se queda sólo con los atributos numéricos (OPCIONAL)
    for col in cdf.columns:
      if cdf[col].dtypes == "object":
          cdf.pop( col )
  # datos de entrada
  X = np.array(cdf.fillna(0.001))
  return X, Y, np.array(cdf.columns)

def convColsNumericas(ndf, atributos_no_convertir = []):
  # hace una copia auxiliar del data frame
  cdf = ndf.copy()
  # convierte todas las no numéricas a numéricas (OPCIONAL)
  for col in cdf.columns:
    if col not in atributos_no_convertir:
      if cdf[col].dtypes == "object":
        # genera diccionario de valores
        valores = cdf[col].unique()
        diccValores = dict(zip(valores, range(len(valores))))
        # realiza el reemplazo
        cdf[col] = cdf[col].map(lambda s: diccValores.get(s) if s in diccValores else s)
  return cdf

# función auxiliar
def generar_estadisticas_detalladas(orDF, titulo=""):
  # título
  print("\n", titulo, ": ")
  # obtiene las estadísticas generales
  estDF = orDF.describe().transpose()
  #  genera y formatea las estadísticas
  if "min" in estDF and "max" in estDF:
    rangoValores = "[ " + estDF["min"].apply('{:.2f}'.format) + " ; " + estDF["max"].apply('{:.2f}'.format) + " ]"
  else:
    rangoValores = estDF["unique"].fillna(0.0).apply('{:.0f}'.format)
  rangoValores.name = "Rango Valores"
  # para campos no numéricos muestra las cantidades por valor
  for col in orDF.columns:
    if orDF[col].dtypes in ("object", "bool"):
      auxStr = str( orDF[col].value_counts() ).replace("\n", " ; ")
      if (auxStr.index("Name")-3) > 0:
        # saca lo del final porque no sirve
        auxStr = auxStr[:auxStr.index("Name")-3]
      rangoValores[col] = "{ " + auxStr + " }"
  if "mean" in estDF and "std" in estDF:
    promValores = estDF["mean"].fillna(0.0).apply('{:.3f}'.format) + " ± " + estDF["std"].fillna(0.0).apply('{:.3f}'.format)
  else:
    promValores = estDF["count"].apply('{:.0f}'.format)
  promValores.name = "Promedio ± Desvío"
  # obtiene valores "ceros" y nulos
  zero_val = (orDF == 0.00).astype(int).sum(axis=0)
  zero_val.name = "¿Valores Ceros?"
  mis_val = orDF.isnull().sum()
  mis_val.name = "¿Valores Nulos?"
  # prepara la nueva tabla para mostrar
  nTable = pd.concat([orDF.dtypes, rangoValores, promValores, zero_val, mis_val], axis=1)
  nTable = nTable.rename( columns = {0: 'Tipo Valor',  1: 'Rango Valores', 2: 'Promedio ± Desvío', 3: '¿Valores Ceros?', 4: '¿Valores Nulos?' } )
  # muestra la nueva tabla
  pd.set_option('max_colwidth', None)
  display(nTable.fillna("-"))
  print("Tiene " + str(orDF.shape[1]) + " atributos y " + str(orDF.shape[0]) + " ejemplos.")
  print("\n")
  return

# muestra las estadísticas
generar_estadisticas_detalladas(df, "> Estadísticas de los datos recolectados")



 > Estadísticas de los datos recolectados : 


,Tipo Valor,Rango Valores,Promedio ± Desvío,¿Valores Ceros?,¿Valores Nulos?
Name,object,{ TIE Defender 2 ; Victory I Class Star Destroyer 2 ; Corellian Gunship 1 ; Y-4 Transport 1 ; Nssis Class Chiss Clawcraft 1 ; .. ; Modular Taskforce Cruiser 1 ; VT49 Decimator Assault Ship 1 ; TIE X2 1 ; TIE Bomber 1 ; AT Barge 1 },107,0,0
Ship Type,object,{ TIE Fighters 20 ; Medium Ships 15 ; Heavy Ships 14 ; TIE Support Craft 10 ; Other Starfighters 10 ; Landing Craft 7 ; Patrol Craft 7 ; Shuttles 6 ; TIE Bombers 6 ; TIE Experimental Craft 5 ; Transporters 4 ; Command Ships 3 },107,0,0
Model,object,{ Corellian Gunship 1 ; TIE/rc Vanguard Fighter 1 ; Y-4 Transport 1 ; Nssis Class Clawcraft 1 ; TIE/ln Fighter 1 ; .. ; Modular Taskforce Cruiser 1 ; VT49 Decimator Assault Ship 1 ; TIE/x2 Fighter 1 ; TIE/bm Bomber 1 ; AT Barge 1 },107,0,0
Manufacturer,object,{ Sienar Fleet Systems 33 ; Kuat Drive Yards 12 ; Cygnus Spaceworks 8 ; Rendili StarDrive 7 ; Sienar Fleet Systems / Imperial Department of Military Research 6 ; Telgorn Corporation 4 ; Santhe / Sienar Technologies 3 ; Mesens Corporation 3 ; Corellian Engineering Corporation 3 ; Republic Sienar Systems 3 ; Telgorn Corporation/KonGar Ship Works 2 ; Meller & Dax 2 ; Incom Corporation 2 ; Sienar Fleet Systems / Zsinj Development Incorporated 2 ; Silviut Corporation 2 ; Sienar Fleet Systems / Shobquix Yards 1 ; Sienar Fleet Systems/Kuat Drive Yards 1 ; Rothana Heavy Engineering / Kuat Drive Yards 1 ; Tagge Industries Shipyards 1 ; Sinar Fleet Systems 1 ; Loronar / Rendili StarDrive /Sienar Fleet Systems and Kuat Drive Yards 1 ; Imperial Shipyards 1 ; Sienar Fleet Systems / Chiss Ascendancy 1 ; Damorian Manufacturing Corporation 1 ; Loronar 1 ; Byss Worx / Imperial Department of Military Research 1 ; Cygnus Spaceworks / Sienar Fleet Systems 1 ; Rothana Heavy Engineering 1 ; SoroSuub Corporation / Cavrilhu Shipyards 1 ; CPG Space Products 1 },107,0,0
Length,object,"{ 7.8 meters 12 ; 6.3 meters 11 ; 6.6 meters 3 ; 153 meters 3 ; 1,600 meters 3 ; 20 meters 3 ; 15 meters 3 ; 9.2 meters 2 ; 598 meters 2 ; 2.1 meters 2 ; 29 meters 2 ; 14.3 meters 2 ; 40 meters 2 ; 38 meters 2 ; 35 meters 2 ; 898 meters 2 ; 6.7 meters 1 ; 180 meters 1 ; 253 meters 1 ; 7.65 meters 1 ; 28 meters 1 ; 250 meters 1 ; 7 meters 1 ; 345 meters 1 ; 119 meters 1 ; 353 meters 1 ; 42 meters 1 ; 18 meters 1 ; 13.8 meters 1 ; 52.5 meters 1 ; 764 meters 1 ; 2.8 meters 1 ; 51 meters 1 ; 75 meters 1 ; 6.1 meters 1 ; 600 meters 1 ; 752 meters 1 ; 21 meters 1 ; 17 meters 1 ; 100 meters 1 ; 8.4 meters 1 ; 15,000 meters 1 ; 43 meters 1 ; 4 meters 1 ; 150 meters 1 ; 16,000 meters (Prototype Lenght: 17,500 meters) 1 ; 7.4 meters 1 ; 24 meters 1 ; 8.7 meters 1 ; 450 meters 1 ; 37 meters 1 ; 501 meters 1 ; 32 meters 1 ; 5 meters 1 ; 4.3 meters 1 ; 230 meters 1 ; 2,500-17,600 meters 1 ; 1,150 meters 1 ; 1,137 meters 1 ; 538 meters 1 ; 298 meters 1 ; 25 meters 1 ; 44 meters 1 ; 11.4 meters 1 ; 15.6 meters 1 ; 300 meters 1 ; 50 meters 1 }",107,0,0
Crew,object,"{ 1 42 ; 2 8 ; 15 3 ; 4 3 ; 3 2 ; 3-5 2 ; 37,085 2 ; 1 fully automated droid brain 2 ; 6 2 ; 5 2 ; 12 2 ; 8 2 ; 275 1 ; 6,107 1 ; 850 1 ; 1,120 1 ; 58 1 ; 91 1 ; 2,807 1 ; 5,200 1 ; 24,724 1 ; 1,092 1 ; 20 1 ; 30-165 (depending upon configuration) 1 ; 906 1 ; Cyborg brain 1 ; 16,210 1 ; 700 1 ; 200 1 ; 150 1 ; 920 1 ; 46 1 ; 36 1 ; 712,645 1 ; 2-4 (Depending on configuration) 1 ; None. (Remote controlled ship) 1 ; 2,112 1 ; 2-4 1 ; 3,505 1 ; 605,745 1 ; 4-6 (Depending on configuration) 1 ; 87,730-619,370 1 ; 1,600-3,700 (varies according to mission profile) 1 ; 7,400 1 ; 3,053 1 ; 3,065 1 ; 45 1 }",107,0,0
Cargo Capacity,object,"{ 65 kilograms 11 ; 150 kilograms 6 ; 500 metric tons 5 ; 100 metric tons 5 ; 45 kilograms 4 ; 200 metric tons 3 ; 1 metric ton 3 ; 100 kilograms 3 ; 300 metric tons 2 ; 2,000 metric tons 2 ; 8,100 metric tons 2 ; 15 metric tons (bomb bay) 2 ; 36,000 metric tons 2 ; 20,000 metric tons 2 ; 75 kilograms 2 ; 50 kilograms 2 ; 400 metric tons 2 ; 80 kilograms 2 ; 6,000 

Tiene 19 atributos y 107 ejemplos.




#Preparar Datos de Naves Imperiales

In [5]:
#@title Preparar Campos donde se extrae parte numérica eliminando texto

# Hace copia de trabajo (para romper datos extraidos)
ndf = df.copy()

# funciones auxiliares
def extraerPrimerNro(columnName):
  global ndf
  # siempre elimina "," como separador de miles
  reemplazar(columnName, ",")
  # toma número con decimales
  ndf[columnName] = ndf[columnName].str.extract('(\d+\.?\d*)\D*', expand=True)
  return

def reemplazar(columnName, oldValue, newValue=""):
  global ndf
  ndf[columnName] = ndf[columnName].str.replace(oldValue, newValue, regex=False)
  ndf[columnName] = ndf[columnName].fillna(0)
  return

def completarVacios(columnName, newValue=-1):
  global ndf
  ndf[columnName] = ndf[columnName].fillna(newValue)
  return

def convertirFloat(columnName):
  global ndf
  ndf[columnName] = ndf[columnName].astype(float)

print("-procesa Length")
reemplazar("Length", " meters")
extraerPrimerNro("Length")
completarVacios("Length", 0)
convertirFloat("Length")

print("-procesa Crew")
extraerPrimerNro("Crew")
completarVacios("Crew", 0)
convertirFloat("Crew")

print("-procesa Troops")
extraerPrimerNro("Troops")
completarVacios("Troops", 0)
convertirFloat("Troops")

print("-procesa Hyperdrive Multiplier")
extraerPrimerNro("Hyperdrive Multiplier")
completarVacios("Hyperdrive Multiplier", -1)
convertirFloat("Hyperdrive Multiplier")

print("-procesa Hyperdrive Backup")
extraerPrimerNro("Hyperdrive Backup")
completarVacios("Hyperdrive Backup", -1)
convertirFloat("Hyperdrive Backup")

print("-procesa Speed")
reemplazar("Speed", " MGLT")
extraerPrimerNro("Speed")
completarVacios("Speed", 0)
convertirFloat("Speed")

print("-procesa Hull")
reemplazar("Hull", " RU")
extraerPrimerNro("Hull")
completarVacios("Hull", 0)
convertirFloat("Hull")

print("-procesa Shields")
reemplazar("Shields", " SBD")
extraerPrimerNro("Shields")
completarVacios("Shields", -1)
convertirFloat("Shields")

# muestra cambios
ndf.head()

-procesa Length
-procesa Crew
-procesa Troops
-procesa Hyperdrive Multiplier
-procesa Hyperdrive Backup
-procesa Speed
-procesa Hull
-procesa Shields


,Name,Ship Type,Model,Manufacturer,Length,Crew,Cargo Capacity,Consumables,Hyperdrive Multiplier,Hyperdrive Backup,Speed,Hull,Shields,Weapons,URL_data,URL_image,Troops,Onboard Craft,Special Features
0,Corellian Gunship,Medium Ships,Corellian Gunship,Corellian Engineering Corporation,119.0,91.0,300 metric tons,8 months,2.00,16.0,29.0,176.0,416.0,"8 Double Turbolaser Cannons, 6 Quad Laser Cannons and 4 Concussion Missile Launchers.",http://insd.swcombine.com/insd/gunship.htm,http://insd.swcombine.com/insd/Ship_files/CORGUNSH.JPG,0.0,NaN,NaN
1,TIE Bomber Trainer,TIE Support Craft,TIE/bmt Bomber Trainer,Sienar Fleet Systems,7.8,1.0,15 metric tons (bomb bay),1 day,-1.00,-1.0,80.0,28.0,-1.0,"2 Laser Cannons, 1 Concussion Missile Launchers and 1 Proton Torpedo, Orbital Mine or Thermal Detonator Launcher.",http://insd.swcombine.com/insd/tiebt.htm,http://insd.swcombine.com/insd/Ship_files/TIEBT.JPG,1.0,NaN,NaN
2,Adz Class Destroyer,Medium Ships,Ads Class Destroyer,Imperial Shipyards,150.0,36.0,400 metric tons,6 months,0.55,12.0,83.0,162.0,600.0,3 Quad Laser Cannon and 3 Ion Cannon Batteries.,http://insd.swcombine.com/insd/adzclass.htm,http://insd.swcombine.com/insd/Ship_files/NOPIC.JPG,8.0,6 TIE Series Fighters.,NaN
3,IPV 1 System Patrol Craft,Medium Ships,IPV 1 System Patrol Craft,Sienar Fleet Systems,153.0,12.0,200 metric tons,3 months,-1.00,-1.0,78.0,285.0,400.0,4 Double Turbolaser Cannons,http://insd.swcombine.com/insd/ipv1.htm,http://insd.swcombine.com/insd/Ship_files/IPV1.JPG,10.0,NaN,NaN
4,TIE Advanced,TIE Fighters,TIE/ad Fighter,Sienar Fleet Systems,7.8,1.0,80 kilograms,5 days,2.00,-1.0,133.0,14.0,40.0,4 Laser Cannons and 2 General Purpose Launchers.,http://insd.swcombine.com/insd/tieadvanc.htm,http://insd.swcombine.com/insd/Ship_files/TIEADVA4.JPG,0.0,NaN,NaN


In [6]:
#@title Preparar campo Cargo Capacity

print("-procesa Cargo Capacity (pasa todo a kilogramos)")

# realiza los cambios considerando tipo de métrica
auxOriList = list(df["Cargo Capacity"])
auxNewList = []
for val in auxOriList:
  #print(val)
  if (val is None) or (str(val) in ["nan", "(varies according to mission profile)"]):
    auxNewList.append( -1 )
  elif str(val)=="Mu-1 and Mu-2: 100 metric tons / Mu-3: 50 metric tons":
    auxNewList.append( 75 )
  else:
    val = val.replace(",", "")
    val = val.replace(" + 1", "")
    if "(" in val:
      posParent = val.index("(")
    else:
      posParent = -1
    if "-" in val:
      # calcula promedio
      posSep1 = val.index("-")
      posSep2 = val.index(" ")
      nval = ( float(val[0:posSep1]) + float(val[posSep1+1:posSep2]) ) / 2
      val = str(nval) + val[posSep2:]
    if "or" in val:
      # calcula promedio
      posSep1 = val.index(" or ")
      posSep2 = val[posSep1+4:].index(" ") + posSep1 + 4
      nval = ( float(val[0:posSep1]) + float(val[posSep1+4:posSep2]) ) / 2
      val = str(nval) + val[posSep2:]
    if "kilograms" in val:
      posAux = val.index("kilograms")
      if (posParent>0) and (posParent<posAux):
        posAux = posParent
      auxNewList.append( float(val[:posAux].strip()) )
    elif "metric ton" in val:
      posAux = val.index("metric ton")
      if (posParent>0) and (posParent<posAux):
        posAux = posParent
      auxNewList.append( float(val[:posAux].strip()) * 1000 )
    else:
      print("No se puede procesar: ", val)

# actualiza los datos
ndf["Cargo Capacity"] = auxNewList
convertirFloat("Cargo Capacity")


ndf.head()

-procesa Cargo Capacity (pasa todo a kilogramos)


,Name,Ship Type,Model,Manufacturer,Length,Crew,Cargo Capacity,Consumables,Hyperdrive Multiplier,Hyperdrive Backup,Speed,Hull,Shields,Weapons,URL_data,URL_image,Troops,Onboard Craft,Special Features
0,Corellian Gunship,Medium Ships,Corellian Gunship,Corellian Engineering Corporation,119.0,91.0,300000.0,8 months,2.00,16.0,29.0,176.0,416.0,"8 Double Turbolaser Cannons, 6 Quad Laser Cannons and 4 Concussion Missile Launchers.",http://insd.swcombine.com/insd/gunship.htm,http://insd.swcombine.com/insd/Ship_files/CORGUNSH.JPG,0.0,NaN,NaN
1,TIE Bomber Trainer,TIE Support Craft,TIE/bmt Bomber Trainer,Sienar Fleet Systems,7.8,1.0,15000.0,1 day,-1.00,-1.0,80.0,28.0,-1.0,"2 Laser Cannons, 1 Concussion Missile Launchers and 1 Proton Torpedo, Orbital Mine or Thermal Detonator Launcher.",http://insd.swcombine.com/insd/tiebt.htm,http://insd.swcombine.com/insd/Ship_files/TIEBT.JPG,1.0,NaN,NaN
2,Adz Class Destroyer,Medium Ships,Ads Class Destroyer,Imperial Shipyards,150.0,36.0,400000.0,6 months,0.55,12.0,83.0,162.0,600.0,3 Quad Laser Cannon and 3 Ion Cannon Batteries.,http://insd.swcombine.com/insd/adzclass.htm,http://insd.swcombine.com/insd/Ship_files/NOPIC.JPG,8.0,6 TIE Series Fighters.,NaN
3,IPV 1 System Patrol Craft,Medium Ships,IPV 1 System Patrol Craft,Sienar Fleet Systems,153.0,12.0,200000.0,3 months,-1.00,-1.0,78.0,285.0,400.0,4 Double Turbolaser Cannons,http://insd.swcombine.com/insd/ipv1.htm,http://insd.swcombine.com/insd/Ship_files/IPV1.JPG,10.0,NaN,NaN
4,TIE Advanced,TIE Fighters,TIE/ad Fighter,Sienar Fleet Systems,7.8,1.0,80.0,5 days,2.00,-1.0,133.0,14.0,40.0,4 Laser Cannons and 2 General Purpose Launchers.,http://insd.swcombine.com/insd/tieadvanc.htm,http://insd.swcombine.com/insd/Ship_files/TIEADVA4.JPG,0.0,NaN,NaN


In [7]:
#@title Preparar campo Consumables

print("-procesa Consumables (pasa todo a meses)")

# realiza los cambios considerando tipo de métrica
auxOriList = list(df["Consumables"])
auxNewList = []
for val in auxOriList:
  #print(val)
  if (val is None) or (str(val) in ["nan", "(varies according to mission profile)"]):
    auxNewList.append( -1 )
  if str(val) == "Mu-1 and Mu-2: 6 months / Mu-3: 2 months":
    auxNewList.append( 4 )
  else:
    val = val.replace(",", "")
    #val = val.replace(" + 1", "")
    if "(" in val:
      posParent = val.index("(")
    else:
      posParent = -1
    if "-" in val:
      # calcula promedio
      posSep1 = val.index("-")
      posSep2 = val.index(" ")
      nval = ( float(val[0:posSep1]) + float(val[posSep1+1:posSep2]) ) / 2
      val = str(nval) + val[posSep2:]
    if "or" in val:
      # calcula promedio
      posSep1 = val.index(" or ")
      posSep2 = val[posSep1+4:].index(" ") + posSep1 + 4
      nval = ( float(val[0:posSep1]) + float(val[posSep1+4:posSep2]) ) / 2
      val = str(nval) + val[posSep2:]
    if "month" in val:
      posAux = val.index("month")
      if (posParent>0) and (posParent<posAux):
        posAux = posParent
      auxNewList.append( float(val[:posAux].strip()) )
    elif "year" in val:
      posAux = val.index("year")
      if (posParent>0) and (posParent<posAux):
        posAux = posParent
      auxNewList.append( float(val[:posAux].strip()) * 12 )
    elif "week" in val:
      posAux = val.index("week")
      if (posParent>0) and (posParent<posAux):
        posAux = posParent
      auxNewList.append( float(val[:posAux].strip()) / 4.5 )
    elif "day" in val:
      posAux = val.index("day")
      if (posParent>0) and (posParent<posAux):
        posAux = posParent
      auxNewList.append( float(val[:posAux].strip()) / 30 )
    else:
      print("No se puede procesar: ", val)

# actualiza los datos
ndf["Consumables"] = auxNewList
convertirFloat("Consumables")

ndf.head()

-procesa Consumables (pasa todo a meses)


,Name,Ship Type,Model,Manufacturer,Length,Crew,Cargo Capacity,Consumables,Hyperdrive Multiplier,Hyperdrive Backup,Speed,Hull,Shields,Weapons,URL_data,URL_image,Troops,Onboard Craft,Special Features
0,Corellian Gunship,Medium Ships,Corellian Gunship,Corellian Engineering Corporation,119.0,91.0,300000.0,8.000000,2.00,16.0,29.0,176.0,416.0,"8 Double Turbolaser Cannons, 6 Quad Laser Cannons and 4 Concussion Missile Launchers.",http://insd.swcombine.com/insd/gunship.htm,http://insd.swcombine.com/insd/Ship_files/CORGUNSH.JPG,0.0,NaN,NaN
1,TIE Bomber Trainer,TIE Support Craft,TIE/bmt Bomber Trainer,Sienar Fleet Systems,7.8,1.0,15000.0,0.033333,-1.00,-1.0,80.0,28.0,-1.0,"2 Laser Cannons, 1 Concussion Missile Launchers and 1 Proton Torpedo, Orbital Mine or Thermal Detonator Launcher.",http://insd.swcombine.com/insd/tiebt.htm,http://insd.swcombine.com/insd/Ship_files/TIEBT.JPG,1.0,NaN,NaN
2,Adz Class Destroyer,Medium Ships,Ads Class Destroyer,Imperial Shipyards,150.0,36.0,400000.0,6.000000,0.55,12.0,83.0,162.0,600.0,3 Quad Laser Cannon and 3 Ion Cannon Batteries.,http://insd.swcombine.com/insd/adzclass.htm,http://insd.swcombine.com/insd/Ship_files/NOPIC.JPG,8.0,6 TIE Series Fighters.,NaN
3,IPV 1 System Patrol Craft,Medium Ships,IPV 1 System Patrol Craft,Sienar Fleet Systems,153.0,12.0,200000.0,3.000000,-1.00,-1.0,78.0,285.0,400.0,4 Double Turbolaser Cannons,http://insd.swcombine.com/insd/ipv1.htm,http://insd.swcombine.com/insd/Ship_files/IPV1.JPG,10.0,NaN,NaN
4,TIE Advanced,TIE Fighters,TIE/ad Fighter,Sienar Fleet Systems,7.8,1.0,80.0,0.166667,2.00,-1.0,133.0,14.0,40.0,4 Laser Cannons and 2 General Purpose Launchers.,http://insd.swcombine.com/insd/tieadvanc.htm,http://insd.swcombine.com/insd/Ship_files/TIEADVA4.JPG,0.0,NaN,NaN


In [8]:
#@title Preparar campo Manufacturer

print("-procesa Manufacturer (asigna codigo valor ID)")

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder().fit(df["Manufacturer"])
ndf["Manufacturer"] = encoder.transform(df["Manufacturer"])
convertirFloat("Manufacturer")

# muestra códificación asignada
print("\n\tCodificación asignada: ")
for i in range(len(encoder.classes_)):
  print("\t\t", i, ":", encoder.classes_[i])
print("")

ndf.head()

-procesa Manufacturer (asigna codigo valor ID)

	Codificación asignada: 
		 0 : Byss Worx / Imperial Department of Military Research
		 1 : CPG Space Products
		 2 : Corellian Engineering Corporation
		 3 : Cygnus Spaceworks
		 4 : Cygnus Spaceworks / Sienar Fleet Systems
		 5 : Damorian Manufacturing Corporation
		 6 : Imperial Shipyards
		 7 : Incom Corporation
		 8 : Kuat Drive Yards
		 9 : Loronar
		 10 : Loronar / Rendili StarDrive /Sienar Fleet Systems and Kuat Drive Yards
		 11 : Meller & Dax
		 12 : Mesens Corporation
		 13 : Rendili StarDrive
		 14 : Republic Sienar Systems
		 15 : Rothana Heavy Engineering
		 16 : Rothana Heavy Engineering / Kuat Drive Yards
		 17 : Santhe / Sienar Technologies
		 18 : Sienar Fleet Systems
		 19 : Sienar Fleet Systems / Chiss Ascendancy
		 20 : Sienar Fleet Systems / Imperial Department of Military Research
		 21 : Sienar Fleet Systems / Shobquix Yards
		 22 : Sienar Fleet Systems / Zsinj Development Incorporated
		 23 : Sienar Fleet Systems/

,Name,Ship Type,Model,Manufacturer,Length,Crew,Cargo Capacity,Consumables,Hyperdrive Multiplier,Hyperdrive Backup,Speed,Hull,Shields,Weapons,URL_data,URL_image,Troops,Onboard Craft,Special Features
0,Corellian Gunship,Medium Ships,Corellian Gunship,2.0,119.0,91.0,300000.0,8.000000,2.00,16.0,29.0,176.0,416.0,"8 Double Turbolaser Cannons, 6 Quad Laser Cannons and 4 Concussion Missile Launchers.",http://insd.swcombine.com/insd/gunship.htm,http://insd.swcombine.com/insd/Ship_files/CORGUNSH.JPG,0.0,NaN,NaN
1,TIE Bomber Trainer,TIE Support Craft,TIE/bmt Bomber Trainer,18.0,7.8,1.0,15000.0,0.033333,-1.00,-1.0,80.0,28.0,-1.0,"2 Laser Cannons, 1 Concussion Missile Launchers and 1 Proton Torpedo, Orbital Mine or Thermal Detonator Launcher.",http://insd.swcombine.com/insd/tiebt.htm,http://insd.swcombine.com/insd/Ship_files/TIEBT.JPG,1.0,NaN,NaN
2,Adz Class Destroyer,Medium Ships,Ads Class Destroyer,6.0,150.0,36.0,400000.0,6.000000,0.55,12.0,83.0,162.0,600.0,3 Quad Laser Cannon and 3 Ion Cannon Batteries.,http://insd.swcombine.com/insd/adzclass.htm,http://insd.swcombine.com/insd/Ship_files/NOPIC.JPG,8.0,6 TIE Series Fighters.,NaN
3,IPV 1 System Patrol Craft,Medium Ships,IPV 1 System Patrol Craft,18.0,153.0,12.0,200000.0,3.000000,-1.00,-1.0,78.0,285.0,400.0,4 Double Turbolaser Cannons,http://insd.swcombine.com/insd/ipv1.htm,http://insd.swcombine.com/insd/Ship_files/IPV1.JPG,10.0,NaN,NaN
4,TIE Advanced,TIE Fighters,TIE/ad Fighter,18.0,7.8,1.0,80.0,0.166667,2.00,-1.0,133.0,14.0,40.0,4 Laser Cannons and 2 General Purpose Launchers.,http://insd.swcombine.com/insd/tieadvanc.htm,http://insd.swcombine.com/insd/Ship_files/TIEADVA4.JPG,0.0,NaN,NaN


In [9]:
#@title Preparar campo Weapons

print("-procesa Weapons (extrae lista de armas y genera nuevas columnas numéricas)")

import re

newValues = []

for f in df["Weapons"]:
  # extrae la lista de armas
  auxSplit =  re.split(", | and ", str(f))
  #print(auxSplit)
  # define columnas en base subcadenas obtenidas
  auxDict = {}
  for w in auxSplit:
    if w=="nan":
      continue
    res = re.search('(\d+\.?\d*)\s(.+)', w)
    if (res is not None):
      ##print(res.group(1), "*", res.group(2))
      auxW = str(res.group(2)).replace(".", "").upper().strip()
      if "(" in auxW:
        # saca toda aclaración entre paréntesis
        auxW = auxW[:auxW.find("(")].strip()
      # si no es plural, le agrega S (para unificar columnas)
      if auxW[len(auxW)-1] != "S":
        auxW = auxW + "S"
      auxQ = float(res.group(1))
    else:
      auxW = w.upper().strip()
      auxQ = 1.0
    if len(auxW)<1:
      auxW = "OTHER"
    if auxW in auxDict:
      auxDict[auxW] = auxDict[auxW] + auxQ
    else:
      auxDict[auxW] = auxQ

  newValues.append( auxDict )


##print(newValues)

# crea nuevo data frame
auxDF = pd.DataFrame.from_dict(newValues)
# completa valores vacios con cero
for col in auxDF:
  auxDF[col] = auxDF[col].fillna(0)
# agrega al data frame anterior y elimina columna original
ndf = pd.concat([ndf, auxDF], axis=1)
ndf = ndf.drop(["Weapons"], axis=1)

ndf.head()

-procesa Weapons (extrae lista de armas y genera nuevas columnas numéricas)


,Name,Ship Type,Model,Manufacturer,Length,Crew,Cargo Capacity,Consumables,Hyperdrive Multiplier,Hyperdrive Backup,Speed,Hull,Shields,URL_data,URL_image,Troops,Onboard Craft,Special Features,DOUBLE TURBOLASER CANNONS,QUAD LASER CANNONS,CONCUSSION MISSILE LAUNCHERS,LASER CANNONS,PROTON TORPEDOS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHER.,ION CANNON BATTERIES,GENERAL PURPOSE LAUNCHERS,HEAVY TURBOLASER BATTERIES,HEAVY LASER CANNONS,ION CANNONS,TRACTOR BEAM PROJECTORS,GRAVITY WELL PROJECTORS,SUPERLASERS,DOUBLE BLASTER CANNON TURRETS,DOUBLE BLASTER CANNONS,DOUBLE LASER CANNONS,TWIN LASER CANNONS,LASER CANNON TURRETS,BUILD-IN SPACE BOMB.,TURBOLASER CANNONS,TURBOLASER BATTERIES,DOUBLE HEAVY LASER TURRETS,BLASTER CANNONS,HEAVY BLASTER CANNONS,(VARIES ACCORDING TO SHIP MODEL),MEDIUM LASER CANNONS,QUAD LASER CANNON TURRETS,MEDIUM TURBOLASERS,HEAVY TURBOLASERS,MEDIUM DUAL TURBOLASERS,PROTON TORPEDO TUBES,TWIN BLASTER CANNON TURRETS,HEAVY TRACTORBEAM PROJECTORS,PROTON TORPEDO LAUNCHERS,HEAVY TURBOLASER CANNONS,MEDIUM ION CANNONS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHERS.,DOUBLE TURBOLASER TURRETS,TURBOLASER CANNON TURRETS,THERMAL DETONATOR LAUNCHERS,QUAD TURBOLASER BATTERIES,DOUBLE TURBOLASER BATTERIES,CUNCUSSION MISSILE LAUNCHERS,ION CANNON TURRETS,REPEATING BLASTER TURRETS,HEAVY TURBOLASER TURRETS,HULL-CUTTING AIRLOCKS,LIGHT LASER CANNONS,DOUBLE LASER CANNON TURRETS,MEDIUM BLASTER CANNONS,LIGHT TURBOLASER CANNONS,HEAVY LASER CANNON TURRETS,QUAD BLASTER CANNON,EMP CANNONS,TURBOLASER TURRETS,MU-1,LASER CANNONS / MU-3: 2 MEDIUM LASER CANNONS,QUAD TURBOLASER CANNONS,QUAD TURBOLASER TURRETS,LASER CANNONS TURRETS
0,Corellian Gunship,Medium Ships,Corellian Gunship,2.0,119.0,91.0,300000.0,8.000000,2.00,16.0,29.0,176.0,416.0,http://insd.swcombine.com/insd/gunship.htm,http://insd.swcombine.com/insd/Ship_files/CORGUNSH.JPG,0.0,NaN,NaN,8.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TIE Bomber Trainer,TIE Support Craft,TIE/bmt Bomber Trainer,18.0,7.8,1.0,15000.0,0.033333,-1.00,-1.0,80.0,28.0,-1.0,http://insd.swcombine.com/insd/tiebt.htm,http://insd.swcombine.com/insd/Ship_files/TIEBT.JPG,1.0,NaN,NaN,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Adz Class Destroyer,Medium Ships,Ads Class Destroyer,6.0,150.0,36.0,400000.0,6.000000,0.55,12.0,83.0,162.0,600.0,http://insd.swcombine.com/insd/adzclass.htm,http://insd.swcombine.com/insd/Ship_files/NOPIC.JPG,8.0,6 TIE Series Fighters.,NaN,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,IPV 1 System Patrol Craft,Medium Ships,IPV 1 System Patrol Craft,18.0,153.0,12.0,200000.0,3.000000,-1.00,-1.0,78.0,285.0,400.0,http://insd.swcombine.com/insd/ipv1.htm,http://insd.swcombine.com/insd/Ship_files/IPV1.JPG,10.0,NaN,NaN,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TIE Advanced,TIE Fighters,TIE/ad Fighter,18.0,7.8,1.0,80.0,0.166667,2.00,-1.0,133.0,14.0,40.0,http://insd.swcombine.com/insd/tieadvanc.htm,http://insd.swcombine.com/insd/Ship_files/TIEADVA4.JPG,0.0,NaN,NaN,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
#@title Preparar campo Special Features

# función auxiliar para clusterizar datos
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

def clusterizarDatosString(colunmnName, cantClusters=5):
  global ndf
  ndf[colunmnName] = ndf[colunmnName].fillna("-")
  documents  = list( ndf[colunmnName] )

  vectorizer = TfidfVectorizer(stop_words='english')
  X = vectorizer.fit_transform(documents)

  model = KMeans(n_clusters=cantClusters, init='k-means++', max_iter=100, n_init=1)
  ndf[colunmnName] = model.fit_transform(X)



print("-procesa Special Features (usando clustering)")

clusterizarDatosString("Special Features")
convertirFloat("Special Features")

ndf.head()

-procesa Special Features (usando clustering)


,Name,Ship Type,Model,Manufacturer,Length,Crew,Cargo Capacity,Consumables,Hyperdrive Multiplier,Hyperdrive Backup,Speed,Hull,Shields,URL_data,URL_image,Troops,Onboard Craft,Special Features,DOUBLE TURBOLASER CANNONS,QUAD LASER CANNONS,CONCUSSION MISSILE LAUNCHERS,LASER CANNONS,PROTON TORPEDOS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHER.,ION CANNON BATTERIES,GENERAL PURPOSE LAUNCHERS,HEAVY TURBOLASER BATTERIES,HEAVY LASER CANNONS,ION CANNONS,TRACTOR BEAM PROJECTORS,GRAVITY WELL PROJECTORS,SUPERLASERS,DOUBLE BLASTER CANNON TURRETS,DOUBLE BLASTER CANNONS,DOUBLE LASER CANNONS,TWIN LASER CANNONS,LASER CANNON TURRETS,BUILD-IN SPACE BOMB.,TURBOLASER CANNONS,TURBOLASER BATTERIES,DOUBLE HEAVY LASER TURRETS,BLASTER CANNONS,HEAVY BLASTER CANNONS,(VARIES ACCORDING TO SHIP MODEL),MEDIUM LASER CANNONS,QUAD LASER CANNON TURRETS,MEDIUM TURBOLASERS,HEAVY TURBOLASERS,MEDIUM DUAL TURBOLASERS,PROTON TORPEDO TUBES,TWIN BLASTER CANNON TURRETS,HEAVY TRACTORBEAM PROJECTORS,PROTON TORPEDO LAUNCHERS,HEAVY TURBOLASER CANNONS,MEDIUM ION CANNONS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHERS.,DOUBLE TURBOLASER TURRETS,TURBOLASER CANNON TURRETS,THERMAL DETONATOR LAUNCHERS,QUAD TURBOLASER BATTERIES,DOUBLE TURBOLASER BATTERIES,CUNCUSSION MISSILE LAUNCHERS,ION CANNON TURRETS,REPEATING BLASTER TURRETS,HEAVY TURBOLASER TURRETS,HULL-CUTTING AIRLOCKS,LIGHT LASER CANNONS,DOUBLE LASER CANNON TURRETS,MEDIUM BLASTER CANNONS,LIGHT TURBOLASER CANNONS,HEAVY LASER CANNON TURRETS,QUAD BLASTER CANNON,EMP CANNONS,TURBOLASER TURRETS,MU-1,LASER CANNONS / MU-3: 2 MEDIUM LASER CANNONS,QUAD TURBOLASER CANNONS,QUAD TURBOLASER TURRETS,LASER CANNONS TURRETS
0,Corellian Gunship,Medium Ships,Corellian Gunship,2.0,119.0,91.0,300000.0,8.000000,2.00,16.0,29.0,176.0,416.0,http://insd.swcombine.com/insd/gunship.htm,http://insd.swcombine.com/insd/Ship_files/CORGUNSH.JPG,0.0,NaN,1.0,8.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TIE Bomber Trainer,TIE Support Craft,TIE/bmt Bomber Trainer,18.0,7.8,1.0,15000.0,0.033333,-1.00,-1.0,80.0,28.0,-1.0,http://insd.swcombine.com/insd/tiebt.htm,http://insd.swcombine.com/insd/Ship_files/TIEBT.JPG,1.0,NaN,1.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Adz Class Destroyer,Medium Ships,Ads Class Destroyer,6.0,150.0,36.0,400000.0,6.000000,0.55,12.0,83.0,162.0,600.0,http://insd.swcombine.com/insd/adzclass.htm,http://insd.swcombine.com/insd/Ship_files/NOPIC.JPG,8.0,6 TIE Series Fighters.,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,IPV 1 System Patrol Craft,Medium Ships,IPV 1 System Patrol Craft,18.0,153.0,12.0,200000.0,3.000000,-1.00,-1.0,78.0,285.0,400.0,http://insd.swcombine.com/insd/ipv1.htm,http://insd.swcombine.com/insd/Ship_files/IPV1.JPG,10.0,NaN,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TIE Advanced,TIE Fighters,TIE/ad Fighter,18.0,7.8,1.0,80.0,0.166667,2.00,-1.0,133.0,14.0,40.0,http://insd.swcombine.com/insd/tieadvanc.htm,http://insd.swcombine.com/insd/Ship_files/TIEADVA4.JPG,0.0,NaN,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
#@title Preparar campo Onboard Craft

print("-procesa Onboard Craft (usando clustering)")

clusterizarDatosString("Onboard Craft")
convertirFloat("Onboard Craft")

ndf.head()

-procesa Onboard Craft (usando clustering)


,Name,Ship Type,Model,Manufacturer,Length,Crew,Cargo Capacity,Consumables,Hyperdrive Multiplier,Hyperdrive Backup,Speed,Hull,Shields,URL_data,URL_image,Troops,Onboard Craft,Special Features,DOUBLE TURBOLASER CANNONS,QUAD LASER CANNONS,CONCUSSION MISSILE LAUNCHERS,LASER CANNONS,PROTON TORPEDOS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHER.,ION CANNON BATTERIES,GENERAL PURPOSE LAUNCHERS,HEAVY TURBOLASER BATTERIES,HEAVY LASER CANNONS,ION CANNONS,TRACTOR BEAM PROJECTORS,GRAVITY WELL PROJECTORS,SUPERLASERS,DOUBLE BLASTER CANNON TURRETS,DOUBLE BLASTER CANNONS,DOUBLE LASER CANNONS,TWIN LASER CANNONS,LASER CANNON TURRETS,BUILD-IN SPACE BOMB.,TURBOLASER CANNONS,TURBOLASER BATTERIES,DOUBLE HEAVY LASER TURRETS,BLASTER CANNONS,HEAVY BLASTER CANNONS,(VARIES ACCORDING TO SHIP MODEL),MEDIUM LASER CANNONS,QUAD LASER CANNON TURRETS,MEDIUM TURBOLASERS,HEAVY TURBOLASERS,MEDIUM DUAL TURBOLASERS,PROTON TORPEDO TUBES,TWIN BLASTER CANNON TURRETS,HEAVY TRACTORBEAM PROJECTORS,PROTON TORPEDO LAUNCHERS,HEAVY TURBOLASER CANNONS,MEDIUM ION CANNONS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHERS.,DOUBLE TURBOLASER TURRETS,TURBOLASER CANNON TURRETS,THERMAL DETONATOR LAUNCHERS,QUAD TURBOLASER BATTERIES,DOUBLE TURBOLASER BATTERIES,CUNCUSSION MISSILE LAUNCHERS,ION CANNON TURRETS,REPEATING BLASTER TURRETS,HEAVY TURBOLASER TURRETS,HULL-CUTTING AIRLOCKS,LIGHT LASER CANNONS,DOUBLE LASER CANNON TURRETS,MEDIUM BLASTER CANNONS,LIGHT TURBOLASER CANNONS,HEAVY LASER CANNON TURRETS,QUAD BLASTER CANNON,EMP CANNONS,TURBOLASER TURRETS,MU-1,LASER CANNONS / MU-3: 2 MEDIUM LASER CANNONS,QUAD TURBOLASER CANNONS,QUAD TURBOLASER TURRETS,LASER CANNONS TURRETS
0,Corellian Gunship,Medium Ships,Corellian Gunship,2.0,119.0,91.0,300000.0,8.000000,2.00,16.0,29.0,176.0,416.0,http://insd.swcombine.com/insd/gunship.htm,http://insd.swcombine.com/insd/Ship_files/CORGUNSH.JPG,0.0,0.020437,1.0,8.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TIE Bomber Trainer,TIE Support Craft,TIE/bmt Bomber Trainer,18.0,7.8,1.0,15000.0,0.033333,-1.00,-1.0,80.0,28.0,-1.0,http://insd.swcombine.com/insd/tiebt.htm,http://insd.swcombine.com/insd/Ship_files/TIEBT.JPG,1.0,0.020437,1.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Adz Class Destroyer,Medium Ships,Ads Class Destroyer,6.0,150.0,36.0,400000.0,6.000000,0.55,12.0,83.0,162.0,600.0,http://insd.swcombine.com/insd/adzclass.htm,http://insd.swcombine.com/insd/Ship_files/NOPIC.JPG,8.0,0.993516,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,IPV 1 System Patrol Craft,Medium Ships,IPV 1 System Patrol Craft,18.0,153.0,12.0,200000.0,3.000000,-1.00,-1.0,78.0,285.0,400.0,http://insd.swcombine.com/insd/ipv1.htm,http://insd.swcombine.com/insd/Ship_files/IPV1.JPG,10.0,0.020437,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TIE Advanced,TIE Fighters,TIE/ad Fighter,18.0,7.8,1.0,80.0,0.166667,2.00,-1.0,133.0,14.0,40.0,http://insd.swcombine.com/insd/tieadvanc.htm,http://insd.swcombine.com/insd/Ship_files/TIEADVA4.JPG,0.0,0.020437,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
#@title Eliminar campos Name, Model y URLs

ndf = ndf.drop(columns=["Name", "Model", "URL_data", "URL_image"])

ndf.head()

,Ship Type,Manufacturer,Length,Crew,Cargo Capacity,Consumables,Hyperdrive Multiplier,Hyperdrive Backup,Speed,Hull,Shields,Troops,Onboard Craft,Special Features,DOUBLE TURBOLASER CANNONS,QUAD LASER CANNONS,CONCUSSION MISSILE LAUNCHERS,LASER CANNONS,PROTON TORPEDOS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHER.,ION CANNON BATTERIES,GENERAL PURPOSE LAUNCHERS,HEAVY TURBOLASER BATTERIES,HEAVY LASER CANNONS,ION CANNONS,TRACTOR BEAM PROJECTORS,GRAVITY WELL PROJECTORS,SUPERLASERS,DOUBLE BLASTER CANNON TURRETS,DOUBLE BLASTER CANNONS,DOUBLE LASER CANNONS,TWIN LASER CANNONS,LASER CANNON TURRETS,BUILD-IN SPACE BOMB.,TURBOLASER CANNONS,TURBOLASER BATTERIES,DOUBLE HEAVY LASER TURRETS,BLASTER CANNONS,HEAVY BLASTER CANNONS,(VARIES ACCORDING TO SHIP MODEL),MEDIUM LASER CANNONS,QUAD LASER CANNON TURRETS,MEDIUM TURBOLASERS,HEAVY TURBOLASERS,MEDIUM DUAL TURBOLASERS,PROTON TORPEDO TUBES,TWIN BLASTER CANNON TURRETS,HEAVY TRACTORBEAM PROJECTORS,PROTON TORPEDO LAUNCHERS,HEAVY TURBOLASER CANNONS,MEDIUM ION CANNONS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHERS.,DOUBLE TURBOLASER TURRETS,TURBOLASER CANNON TURRETS,THERMAL DETONATOR LAUNCHERS,QUAD TURBOLASER BATTERIES,DOUBLE TURBOLASER BATTERIES,CUNCUSSION MISSILE LAUNCHERS,ION CANNON TURRETS,REPEATING BLASTER TURRETS,HEAVY TURBOLASER TURRETS,HULL-CUTTING AIRLOCKS,LIGHT LASER CANNONS,DOUBLE LASER CANNON TURRETS,MEDIUM BLASTER CANNONS,LIGHT TURBOLASER CANNONS,HEAVY LASER CANNON TURRETS,QUAD BLASTER CANNON,EMP CANNONS,TURBOLASER TURRETS,MU-1,LASER CANNONS / MU-3: 2 MEDIUM LASER CANNONS,QUAD TURBOLASER CANNONS,QUAD TURBOLASER TURRETS,LASER CANNONS TURRETS
0,Medium Ships,2.0,119.0,91.0,300000.0,8.000000,2.00,16.0,29.0,176.0,416.0,0.0,0.020437,1.0,8.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TIE Support Craft,18.0,7.8,1.0,15000.0,0.033333,-1.00,-1.0,80.0,28.0,-1.0,1.0,0.020437,1.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Medium Ships,6.0,150.0,36.0,400000.0,6.000000,0.55,12.0,83.0,162.0,600.0,8.0,0.993516,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Medium Ships,18.0,153.0,12.0,200000.0,3.000000,-1.00,-1.0,78.0,285.0,400.0,10.0,0.020437,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TIE Fighters,18.0,7.8,1.0,80.0,0.166667,2.00,-1.0,133.0,14.0,40.0,0.0,0.020437,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
#@title Reordena columnas para que Ship Type quede al final

ndf = ndf.reindex(columns=(list([a for a in ndf.columns if a != 'Ship Type'] + ['Ship Type']) ))

ndf.head()

,Manufacturer,Length,Crew,Cargo Capacity,Consumables,Hyperdrive Multiplier,Hyperdrive Backup,Speed,Hull,Shields,Troops,Onboard Craft,Special Features,DOUBLE TURBOLASER CANNONS,QUAD LASER CANNONS,CONCUSSION MISSILE LAUNCHERS,LASER CANNONS,PROTON TORPEDOS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHER.,ION CANNON BATTERIES,GENERAL PURPOSE LAUNCHERS,HEAVY TURBOLASER BATTERIES,HEAVY LASER CANNONS,ION CANNONS,TRACTOR BEAM PROJECTORS,GRAVITY WELL PROJECTORS,SUPERLASERS,DOUBLE BLASTER CANNON TURRETS,DOUBLE BLASTER CANNONS,DOUBLE LASER CANNONS,TWIN LASER CANNONS,LASER CANNON TURRETS,BUILD-IN SPACE BOMB.,TURBOLASER CANNONS,TURBOLASER BATTERIES,DOUBLE HEAVY LASER TURRETS,BLASTER CANNONS,HEAVY BLASTER CANNONS,(VARIES ACCORDING TO SHIP MODEL),MEDIUM LASER CANNONS,QUAD LASER CANNON TURRETS,MEDIUM TURBOLASERS,HEAVY TURBOLASERS,MEDIUM DUAL TURBOLASERS,PROTON TORPEDO TUBES,TWIN BLASTER CANNON TURRETS,HEAVY TRACTORBEAM PROJECTORS,PROTON TORPEDO LAUNCHERS,HEAVY TURBOLASER CANNONS,MEDIUM ION CANNONS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHERS.,DOUBLE TURBOLASER TURRETS,TURBOLASER CANNON TURRETS,THERMAL DETONATOR LAUNCHERS,QUAD TURBOLASER BATTERIES,DOUBLE TURBOLASER BATTERIES,CUNCUSSION MISSILE LAUNCHERS,ION CANNON TURRETS,REPEATING BLASTER TURRETS,HEAVY TURBOLASER TURRETS,HULL-CUTTING AIRLOCKS,LIGHT LASER CANNONS,DOUBLE LASER CANNON TURRETS,MEDIUM BLASTER CANNONS,LIGHT TURBOLASER CANNONS,HEAVY LASER CANNON TURRETS,QUAD BLASTER CANNON,EMP CANNONS,TURBOLASER TURRETS,MU-1,LASER CANNONS / MU-3: 2 MEDIUM LASER CANNONS,QUAD TURBOLASER CANNONS,QUAD TURBOLASER TURRETS,LASER CANNONS TURRETS,Ship Type
0,2.0,119.0,91.0,300000.0,8.000000,2.00,16.0,29.0,176.0,416.0,0.0,0.020437,1.0,8.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Medium Ships
1,18.0,7.8,1.0,15000.0,0.033333,-1.00,-1.0,80.0,28.0,-1.0,1.0,0.020437,1.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TIE Support Craft
2,6.0,150.0,36.0,400000.0,6.000000,0.55,12.0,83.0,162.0,600.0,8.0,0.993516,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Medium Ships
3,18.0,153.0,12.0,200000.0,3.000000,-1.00,-1.0,78.0,285.0,400.0,10.0,0.020437,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Medium Ships
4,18.0,7.8,1.0,80.0,0.166667,2.00,-1.0,133.0,14.0,40.0,0.0,0.020437,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TIE Fighters


In [14]:
#@title Mostrar estadísticas de datos preparados

generar_estadisticas_detalladas(ndf, "> Estadísticas de los datos preparados")



 > Estadísticas de los datos preparados : 


,Tipo Valor,Rango Valores,Promedio ± Desvío,¿Valores Ceros?,¿Valores Nulos?
Manufacturer,float64,[ 0.00 ; 29.00 ],14.626 ± 7.205,1,0
Length,float64,[ 2.10 ; 16000.00 ],481.423 ± 2123.802,0,0
Crew,float64,[ 0.00 ; 712645.00 ],14605.710 ± 90254.128,2,0
Cargo Capacity,float64,[ -1.00 ; 600000000.00 ],15213112.766 ± 75767701.080,0,0
Consumables,float64,[ 0.03 ; 120.00 ],8.328 ± 19.252,0,0
Hyperdrive Multiplier,float64,[ -1.00 ; 15.00 ],0.899 ± 2.004,0,0
Hyperdrive Backup,float64,[ -1.00 ; 20.00 ],4.972 ± 7.145,0,0
Speed,float64,[ 4.00 ; 155.00 ],69.159 ± 38.796,0,0
Hull,float64,[ 5.00 ; 77710.00 ],1473.692 ± 8888.793,0,0
Shields,float64,[ -1.00 ; 136000.00 ],2713.785 ± 15989.261,0,0


Tiene 75 atributos y 107 ejemplos.




#Exportar datos de Naves Imperiales

In [15]:
#@title Acceder al Drive

# monta Google Drive:
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive')

# directorio local en Google Drive
path = '/content/gdrive/My Drive/IA/demoStarWars/datos/'  #@param {type:"string"}


Mounted at /content/gdrive


In [16]:
#@title Exporta los datos como CSV

download_CSV = False #@param{type:"boolean"}

import os

# crea directorio si no existe
if not os.path.isdir(path):
  os.mkdir(path)

def data2CSV(df, nomArch, descripcion):
  if df is None:
    print("No hay " + descripcion + " para exportar")
  df.to_csv(nomArch, index=False)
  if download_CSV:
    files.download(nomArch)
  print(descripcion + " exportados como ", nomArch)

# exporta datos
data2CSV(df, path + "navesOri.csv", "Datos originales")
data2CSV(ndf, path + "naves.csv", "Datos preparados")


Datos originales exportados como  /content/gdrive/My Drive/IA/demoStarWars/datos/navesOri.csv
Datos preparados exportados como  /content/gdrive/My Drive/IA/demoStarWars/datos/naves.csv
